# モデルサーバーを使用した推論の実行

このノートブックでは、gRPC 推論エンドポイントを使用してモデルをクエリします。

## まず、プレースホルダー（下の `replace_me` ）を、前の手順でモデル提供構成から取得した grpcUrl に置き換えます。

ポート、モデル名、クラス定義ファイルはすでに入力されています。

In [ ]:
grpc_url = 'replace_me' # e.g. 'grpc://modelmesh-serving.user1-data-science-project:8033'
model_name = 'coolstore'
classes_file = 'classes.yaml'

## 必要なライブラリがあることを確認
必要に応じて、requirements.txt を見て、どのライブラリが使用されているかを確認することができます。そのうちの 1 つは PyTorch です。これはモデルのトレーニングに使用されたフレームワークです。このライブラリの一部では、モデルをクエリする前に画像の準備を行う必要があります。

ライブラリのインストールには最大 1 分かかる場合があります。しばらくお待ちください。 **このインストール中にエラーや警告が表示される場合がありますが、無視して構いません** 。

In [ ]:
!pip install -qr requirements.txt

# CPU のみのバージョンの Pytorch をインストール
!pip install -q -r requirements-torch-cpu.txt

## 推論を行うためのすべてのヘルパー コードが含まれるライブラリとモジュールをインポート

In [ ]:
import re
import cv2
from matplotlib import pyplot as plt
from PIL import Image

from remote_infer_grpc import ort_v5

また、ファイル `remote_infer_grpc.py` を参照して、推論エンドポイントにデータを送信する前後にデータで発生する前処理と後処理の詳細を確認することもできます。

## 推論用のパラメータを設定

In [ ]:
# 1. 解析したい画像
image_path='images/RHODS_cool_store.png' # You can replace this with an image you upload

# 2. 信頼度のしきい値、0 ～ 1 (スコアが低い検出は保持されません)
conf = 0.4

# 3. 結合しきい値を超える交差、0 から 1 の間 (重複するボックスをクリーンアップ)
iou = 0.6

## 推論を開始して結果を表示

In [ ]:

# ホストを抽出
pattern1 = r"\/\/(.+?):\d+"
match1 = re.search(pattern1, grpc_url)
if match1:
    grpc_host = match1.group(1)

# ポートを抽出
pattern2 = r"(\d+)$"
match2 = re.search(pattern2, grpc_url)
if match2:
    grpc_port = match2.group(1)

infer=ort_v5(grpc_host, grpc_port, model_name, 640, classes_file)
img, out, result = infer(image_path, conf, iou)
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img);

完璧です。モデル提供 API が機能していることがわかります。

これで、インストラクションに戻ります。